In [10]:
import math
import numpy as np
import random
# from scipy.stats import norm
from itertools import combinations
from scipy.stats import norm


In [11]:


def dynamic_programming_algorithm(X, UD):
    """_summary_

    Args:
        X (_type_): _description_
        UD (_type_): _description_

    Returns:
        _type_: _description_
    """
    f_X = np.zeros(len(UD) + 1)
    f_X[0] = 1

    for T_i in UD:
        p_Xi = get_probability_in_transaction(X, T_i)
        f_X_prime = np.zeros_like(f_X)
        f_X_prime[0] = (1 - p_Xi) * f_X[0]

        for k in range(1, len(UD) + 1):
            f_X_prime[k] = p_Xi * f_X[k - 1] + (1 - p_Xi) * f_X[k]
        f_X = f_X_prime

    return f_X

def DC(X, PDB):
    n = len(PDB)
    
    # Base case
    if n == 1:
        prob = get_probability_in_transaction(X, PDB[0])
        f_X = np.zeros(2)
        f_X[0] = 1 - prob
        f_X[1] = prob
        return f_X

    # Recursive case: Partition PDB
    mid = n // 2
    D1 = PDB[:mid]
    D2 = PDB[mid:]

    # Recursively call DC on partitions
    f_X1 = DC(X, D1)
    f_X2 = DC(X, D2)

    # Adjust length of f_X1 and f_X2 for convolution
    len_total = len(f_X1) + len(f_X2) - 1
    f_X1 = np.pad(f_X1, (0, len_total - len(f_X1)), mode='constant')
    f_X2 = np.pad(f_X2, (0, len_total - len(f_X2)), mode='constant')

    # Convolution using FFT
    f_X = np.fft.ifft(np.fft.fft(f_X1) * np.fft.fft(f_X2)).real
    f_X = f_X[:n+1]  # Take only the first n+1 elements
    f_X = np.round(f_X, decimals=10)  # Fix numerical inaccuracies

    return f_X

def get_probability_in_transaction(X, T):
    prob = 1.0
    for i in X:
        prob *= T.get(i, 0.0)
    return prob

def get_weighted_itemset(X, W):
    weighted = 0

    for i in X:
        weighted += W.get(i, 0.0)

    return weighted / len(X)

In [12]:
# def compute_prF(X, UD, wfpt):
#     convolution_vector = dynamic_programming_algorithm(X, UD)
#     prF = 0

#     for i in range(len(UD), 0, -1):
#         prF += convolution_vector[i]

#         if prF > wfpt:
#             return i

#     return -1

def compute_prWF(X, UD, W, wfpt):
    convolution_vector = DC(X, UD)
    prWF = 0

    for i in range(len(UD), 0, -1):
        prWF += convolution_vector[i]*get_weighted_itemset(X, W)

        if prWF > wfpt:
            return i

    return -1

# def compute_prMWF(X, UD, W, min_sup):
#     max_weighted = 0

#     for i in X:
#         if W.get(i) > max_weighted:
#             max_weighted = W.get(i)

#     prF = compute_prF(X, UD, min_sup)

#     return max_weighted * prF

def compute_expected_support(X, UD):
    expected_support = 0

    for Ti in UD:
        expected_support += get_probability_in_transaction(X, Ti)

    return expected_support

def compute_approximate_prWF(X, UD, W, wfpt):
    expectation = compute_expected_support(X, UD)
    variance = compute_variance(X, UD)
    
    icdf_value = norm.ppf(1 - wfpt/get_weighted_itemset(X, W), loc=expectation, scale=np.sqrt(variance))
    
    return icdf_value * np.sqrt(variance) + expectation

def compute_variance(X, UD):
    var = 0

    for Ti in UD:
        probX= get_probability_in_transaction(X, Ti)
        var += probX * (1 - probX)

    return var

def lower_bound_expected_support(min_sup, min_conf):
    return (2*min_sup - np.log(min_conf) - np.sqrt((np.log(min_conf))**2 - 8*min_sup*np.log(min_conf))) / 2

def upper_bound_expected_support(min_sup, min_conf):
    return min_sup - np.log(1 - min_conf) + np.sqrt(np.log(1 - min_conf)**2 - 2*min_sup*np.log(1 - min_conf))

In [13]:
def discover_single_items(UD):
    # Implement discovery of single items using Chernoff-Hoeffding bound
    single_items = set()

    for Ti in UD:
        for item in Ti.keys():
            single_items.add(item)

    return single_items

def FM(min_sup, min_conf, expectation, variance):
  if expectation > upper_bound_expected_support(min_sup, min_conf):
    return True
  else:
    # normal_distribution = norm(loc=expectation, scale=np.sqrt(variance))
    # return 1 - normal_distribution.cdf((min_sup - expectation)/np.sqrt(variance))
    icdf_value = norm.ppf(1 - min_conf, loc=expectation, scale=np.sqrt(variance))
    return icdf_value * np.sqrt(variance) + expectation


def candidate_generate_expected_bound(UD, W, min_sup, min_conf):
    listCandidate = []
    i = 1
    L = discover_single_items(UD)

    while True:
        Ci = []

        combinations_list = list(combinations(L, i))
        print("combinations_list", combinations_list)

        for item in combinations_list:
            E = 0
            Var = 0
            count = 0

            for j in range(0, len(UD)):
                Tj = UD[j]
                # item = list(item)
                probX = get_probability_in_transaction(item, Tj)
                weightedX = get_weighted_itemset(item, W)
                
                # if (weightedX <= 0):
                #     weightedX = 1
                # print("probX", probX)

                if probX > 0:
                    E += probX
                    Var += probX * (1 - probX)
                    count += 1

                # print("Itemset", item)
                # print("Expectation", E)
                # print("Lower bound", lower_bound_expected_support(min_sup, min_conf / weightedX))
                # print()

                    if E  >= lower_bound_expected_support(min_sup, min_conf / weightedX) and count >= min_sup:
                        Ci.append((item, E, Var, j))
                        
                        break

        i += 1
        listCandidate.append(Ci)
        # print("Ci",Ci)
        # print("L", L)

        L = set()

        for c in Ci:
            for item in c[0]:
              L.add(item)

        if len(L) == 0:
            return listCandidate


# Implement algorithms

In [14]:
def apfi_max(UD, W, min_sup, min_conf):
    """
    Perform the APFI-MAX algorithm on the given uncertain database.

    Parameters:
    UD (list): Uncertain Database
    T (float): Minimum support threshold
    τ (float): Minimum probability threshold

    Returns:
    list: Container RES for PMFI
    """

    # Step to obtain candidates with algorithm 1 is assumed to be done separately
    C = candidate_generate_expected_bound(UD, W, min_sup, min_conf)

    Fre_Pre = []
    Fre_Cur = []
    RES = []

    N = len(C)

    for i in range(N, 0, -1):
        for j in range(len(C[i-1])):
            X = C[i-1][j]
            # # print(X)
            if X in Fre_Pre and all(item in X for item in Fre_Pre):
                Fre_Cur.append(X)
                continue
            # measure_frequency = FM(min_sup, min_conf/get_weighted_itemset(X[0], W), X[1], X[2]) 
            # Call algorithm 3 to measure its frequency (assuming we have a function for that)
            # if measure_frequency:
            #     isContain = False
            #     for y in RES:
            #         if set(X[0]).issubset(set(y)):
            #             isContain = True
            #             break
            #     if not isContain:
            #         RES.append(list(X[0]))
            #     Fre_Cur.append(X)
            # else:
            #     if measure_frequency >= min_sup:
            #     # RES.append(X)
            #     # Fre_Cur.append(X)
            #         isContain = False
            #         for y in RES:
            #             if set(X[0]).issubset(set(y)):
            #                 isContain = True
            #                 break
            #         if not isContain:
            #             RES.append(list(X[0]))
            #         Fre_Cur.append(X)
            # print(X[0], "--- ", compute_prF(X[0], UD, min_conf))
            
            # if X[1] > upper_bound_expected_support(min_sup, min_conf/get_weighted_itemset(X[0], W)):
            #     isContain = False
            #     for y in RES:
            #         if set(X[0]).issubset(set(y)):
            #             isContain = True
            #             break
            #     if not isContain:
            #         RES.append(list(X[0]))
            #     Fre_Cur.append(X)
            #     continue
            
            if compute_prWF(X[0], UD, W, min_conf) >= min_sup:
                isContain = False
                for y in RES:
                    if set(X[0]).issubset(set(y)):
                        isContain = True
                        break
                if not isContain:
                    RES.append(list(X[0]))
                Fre_Cur.append(X)

        Fre_Pre = Fre_Cur
        Fre_Cur = []

    return RES




# Example

In [15]:
# Example 1
UD = [
    {'A': 0.6, 'B': 0.7},
    {'A': 0.2, 'C': 0.3},
]

weighted = {
    'A': 1,
    'B': 1,
    'C': 1
}

min_sup = 1
wfpt = 0.1

# #Example 2
UD = [
    {"A": 0.5, "B": 0.7, "D": 0.8, "E": 0.9},
    {"B": 0.6, "C": 0.8, "D": 0.6, "E": 0.8},
    {"C": 0.6, "D": 0.9, "E": 0.5},
    {"A": 0.6, "C": 0.7, "D": 0.8, "E": 0.8},
    {"A": 0.8, "B": 0.9, "C": 0.5, "D": 0.6, "E": 0.7},
    {"B": 0.6, "D": 0.9, "E": 0.8},
]

weighted = {
    "A": 0.3,
    "B": 0.9,
    "C": 0.5,
    "D": 0.6,
    "E": 0.9
}

min_sup = 2
wfpt = 0.2
# print(candidate_generate_expected_bound(UD, min_sup, wfpt))
print(apfi_max(UD, weighted, min_sup, wfpt))

combinations_list [('A',), ('D',), ('B',), ('C',), ('E',)]
combinations_list [('A', 'D'), ('A', 'B'), ('A', 'C'), ('A', 'E'), ('D', 'B'), ('D', 'C'), ('D', 'E'), ('B', 'C'), ('B', 'E'), ('C', 'E')]
combinations_list [('A', 'D', 'B'), ('A', 'D', 'C'), ('A', 'D', 'E'), ('A', 'B', 'C'), ('A', 'B', 'E'), ('A', 'C', 'E'), ('D', 'B', 'C'), ('D', 'B', 'E'), ('D', 'C', 'E'), ('B', 'C', 'E')]
combinations_list [('A', 'D', 'B', 'C'), ('A', 'D', 'B', 'E'), ('A', 'D', 'C', 'E'), ('A', 'B', 'C', 'E'), ('D', 'B', 'C', 'E')]
combinations_list [('A', 'D', 'B', 'C', 'E')]
[['D', 'B', 'E'], ['D', 'C', 'E'], ['A', 'E']]


# Real dataset

In [16]:
def read_dataset(file_path, mean, variance):
    uncertain_database = []

    # Initialize normal distribution
    random.seed(12345)

    try:
        with open(file_path, 'r') as file:
            curr_id_transaction = 0
            count_num_line = 1

            for line in file:
                data_line_transaction = line.strip().split(" ")
                cur_transaction = {}

                for item in data_line_transaction:
                    data_item = item.split("-")
                    value = data_item[0]
                    prob = float(data_item[1])
                    cur_transaction[value] = prob

                uncertain_database.append(cur_transaction)
                curr_id_transaction += 1
                count_num_line += 1
    except IOError as e:
        print(e)

    print(uncertain_database)

    return uncertain_database

def generate_weighted_table(uncertain_database):
    weighted_table = {}
    distinct_itemset_database = discover_single_items(uncertain_database)

    random.seed(12345)

    for item in distinct_itemset_database:
        weighted = round(random.random(), 1)
        if weighted <= 0:
            weighted = 0.1
        else:
            if weighted >= 1:
                weighted = 0.9
        weighted_table[item] = weighted

    return weighted_table

In [17]:
path = './data/accidents_10K.data'

UD = read_dataset(path, 0.78, 0.65)
W = generate_weighted_table(UD)

for i in range(0, 5):
    print(UD[i])

[{'1': 0.4, '2': 0.7, '3': 0.8, '4': 0.3, '5': 0.1, '6': 0.3, '7': 0.2, '8': 0.9, '9': 0.3, '10': 0.7, '11': 1.0, '12': 0.5, '13': 0.1, '14': 0.9, '15': 0.7, '16': 0.4, '17': 0.2, '18': 0.5, '19': 0.2, '20': 0.3, '21': 0.5, '22': 0.4, '23': 0.4, '24': 0.7, '25': 0.3, '26': 0.0, '27': 0.2, '28': 0.6, '29': 0.1, '30': 0.5, '31': 0.6}, {'2': 0.8, '5': 0.5, '7': 0.7, '8': 0.1, '9': 0.9, '10': 1.0, '12': 0.5, '13': 0.8, '14': 0.9, '15': 0.6, '16': 0.2, '17': 0.4, '18': 0.1, '20': 0.2, '22': 0.1, '23': 0.9, '24': 0.9, '25': 0.1, '27': 0.7, '28': 0.9, '29': 1.0, '32': 0.9, '33': 0.8, '34': 0.6, '35': 0.9, '36': 1.0, '37': 0.6, '38': 0.4, '39': 0.8}, {'7': 0.6, '10': 0.7, '12': 0.9, '13': 0.9, '14': 0.1, '15': 0.1, '16': 0.3, '17': 0.1, '18': 0.5, '20': 0.3, '25': 0.8, '28': 0.6, '29': 0.1, '30': 0.9, '33': 0.2, '40': 0.5, '41': 0.5, '42': 0.3, '43': 0.5, '44': 0.7, '45': 0.1, '46': 0.4, '47': 0.4, '48': 0.7, '49': 0.5, '50': 0.4, '51': 0.9, '52': 0.8}, {'1': 0.3, '5': 0.6, '8': 0.6, '10': 0.6

In [18]:
import time

# Start the timer
start_time = time.time()

min_sup = 0.1*10000
wfpt = 0.6
# Your code here
result = apfi_max(UD, W, min_sup, wfpt)
print(result)
print(len(result))

# End the timer
end_time = time.time()

# Calculate the runtime
runtime = end_time - start_time
print(f"Runtime: {runtime} seconds")


combinations_list [('285',), ('196',), ('160',), ('227',), ('129',), ('48',), ('82',), ('87',), ('174',), ('96',), ('231',), ('157',), ('183',), ('274',), ('32',), ('244',), ('275',), ('301',), ('156',), ('58',), ('175',), ('125',), ('176',), ('215',), ('201',), ('262',), ('23',), ('51',), ('21',), ('207',), ('85',), ('119',), ('110',), ('185',), ('229',), ('214',), ('248',), ('267',), ('210',), ('250',), ('280',), ('257',), ('195',), ('117',), ('251',), ('186',), ('190',), ('198',), ('146',), ('150',), ('38',), ('225',), ('74',), ('291',), ('224',), ('106',), ('120',), ('84',), ('52',), ('296',), ('168',), ('179',), ('256',), ('101',), ('284',), ('208',), ('171',), ('269',), ('243',), ('155',), ('148',), ('6',), ('154',), ('142',), ('281',), ('221',), ('39',), ('12',), ('113',), ('261',), ('66',), ('260',), ('10',), ('42',), ('108',), ('307',), ('134',), ('132',), ('277',), ('2',), ('75',), ('116',), ('242',), ('35',), ('226',), ('131',), ('70',), ('128',), ('203',), ('246',), ('40',)

C:\Users\trant\AppData\Local\Temp\ipykernel_2756\1759249285.py:62: RuntimeWarning: invalid value encountered in sqrt
  return (2*min_sup - np.log(min_conf) - np.sqrt((np.log(min_conf))**2 - 8*min_sup*np.log(min_conf))) / 2


combinations_list [('64', '17'), ('64', '7'), ('64', '14'), ('64', '82'), ('64', '22'), ('64', '16'), ('64', '30'), ('64', '43'), ('64', '41'), ('64', '102'), ('64', '53'), ('64', '59'), ('64', '12'), ('64', '1'), ('64', '66'), ('64', '28'), ('64', '61'), ('64', '68'), ('64', '33'), ('64', '108'), ('64', '23'), ('64', '80'), ('17', '7'), ('17', '14'), ('17', '82'), ('17', '22'), ('17', '16'), ('17', '30'), ('17', '43'), ('17', '41'), ('17', '102'), ('17', '53'), ('17', '59'), ('17', '12'), ('17', '1'), ('17', '66'), ('17', '28'), ('17', '61'), ('17', '68'), ('17', '33'), ('17', '108'), ('17', '23'), ('17', '80'), ('7', '14'), ('7', '82'), ('7', '22'), ('7', '16'), ('7', '30'), ('7', '43'), ('7', '41'), ('7', '102'), ('7', '53'), ('7', '59'), ('7', '12'), ('7', '1'), ('7', '66'), ('7', '28'), ('7', '61'), ('7', '68'), ('7', '33'), ('7', '108'), ('7', '23'), ('7', '80'), ('14', '82'), ('14', '22'), ('14', '16'), ('14', '30'), ('14', '43'), ('14', '41'), ('14', '102'), ('14', '53'), ('14'